<a href="https://colab.research.google.com/github/SupritiVijay/Question-Answering-Based-System/blob/main/NLP_Question_and_Answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import pandas as pd
import json

In [15]:
#load json
def load_json(URL):

  df_data = pd.read_json(URL)
  final_data = pd.json_normalize(df_data.data)

  #unwind json file
  questions = []
  answers = []
  for paragraph in final_data.paragraphs:
    for qas in paragraph:
      for question in qas['qas']:
        for ans in question['answers']:
          questions.append(question['question'])
          answers.append(ans['text'])

  #create dataframe
  df = pd.DataFrame(list(zip(questions, answers)),
               columns =['questions', 'answers']) 
  
  return df

In [16]:
#Stemming
from nltk.stem import PorterStemmer
import re
from tqdm.notebook import tqdm

ps = PorterStemmer()

def preprocess_qs(sentence):
  sentence = re.sub("[^a-zA-Z]", " ", sentence)
  return ' '.join([ps.stem(word) for word in sentence.split() if word!=''])



def preprocess_entiredata(data):
  for i in tqdm(range(len(data))):
    data[i] = preprocess_qs(data[i])
  
  return 0

In [17]:
def embedding_matrix(questions, vocab):
  embedding = [] 
  for sentence in tqdm(questions):
    embedding_single=[]
    for i in sentence.split():
      if(i not in vocab):
        vocab.append(i)
      embedding_single.append(vocab.index(i))
    
    embedding.append(embedding_single)  

  print(embedding)

  return embedding, vocab

In [18]:
#Training Set
df = load_json('https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json')
preprocess_entiredata(df.questions)

vocab = []
train_questions_embeddings,vocab = embedding_matrix(df.questions, vocab)

  0%|          | 0/86821 [00:00<?, ?it/s]

  0%|          | 0/86821 [00:00<?, ?it/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [58]:
# train = ['what am I doing', 'what can I do']
test_sentence = ['In what citi and state did beyonc grow up','What is dukkha and the second truth?']

In [59]:
#Test Set preprocessing and creation of embedding matrix
from tqdm.notebook import tqdm

preprocess_entiredata(test_sentence)
test_embedding, _ = embedding_matrix(test_sentence,vocab)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

[[22, 6, 23, 18, 24, 1, 2, 12, 13], [6, 93, 4042, 18, 31, 60, 3978]]


In [82]:
#Checking for similar questions
def similiar_questions(train_embedding,test_embedding):
  similiar = []
  main_similiar = []

  for i in test_embedding:
    similiar = []
    i = list(set(i))
    for j in train_embedding:
      j = list(set(j))
      f = 0
      for x in i:
        flag = False
        for y in j:
          if x==y:
            flag = True
            break
        if flag:
          f += 1
      similiar.append(f)
      
    # print(similiar)
      
    main_similiar.append(similiar.index(max(similiar)))

  answers = []
  for i,index in enumerate(main_similiar):
    answers.append(df.answers[index])
    print('Test Question:', test_sentence[i],'\nOriginal Question:',df.questions[index],'\nAnswer:', tentative_answers[i])
    print('\n')

  return answers

In [83]:
tentative_answers = similiar_questions(train_questions_embeddings, test_embedding)

Test Question: In what citi and state did beyonc grow up 
Original Question: In what citi and state did beyonc grow up 
Answer: Houston, Texas


Test Question: what is dukkha and the second truth 
Original Question: what is the second truth 
Answer: dukkha can be known.


